In [1]:
import pandas as pd
from pathlib import Path

In [2]:
def load_data_no_folds(scenario_dir_path, dataset_type):
    # make dict to store data
    storage_list = list()
    # make paths for the specified dataset
    train_annotations_dir = Path(scenario_dir_path, dataset_type, "annotations")
    train_physiology_dir = Path(scenario_dir_path, dataset_type, "physiology")
    # sort contents of dirs, so that physiology and annotations are in the same order  
    train_physiology_files = sorted(Path(train_physiology_dir).iterdir())
    train_annotation_files = sorted(Path(train_annotations_dir).iterdir())
    # iterate over annotation and physiology files
    for physiology_file_path, annotations_file_path in zip(train_physiology_files, train_annotation_files):
        # make sure that we load corresponding physiology and annotations
        assert physiology_file_path.name == annotations_file_path.name, "Order mismatch"
        # load data from files
        df_physiology = pd.read_csv(physiology_file_path, index_col="time")
        df_annotations = pd.read_csv(annotations_file_path, index_col="time")
        # store data
        storage_list.append((annotations_file_path.name, df_physiology, df_annotations))
    return storage_list

In [3]:
def load_data_with_folds(scenario_dir_path, dataset_type):
    # make dict to store data
    storage_dict = dict()
    # iterate over the scenario directory
    for fold_dir in Path(scenario_dir_path).iterdir():
        # make paths for current fold
        train_annotations_dir = Path(fold_dir, f"{dataset_type}/annotations/")
        train_physiology_dir = Path(fold_dir, f"{dataset_type}/physiology/")
        # make key in a dict for current fold 
        storage_dict.setdefault(fold_dir.name, list())
        # sort contents of dirs, so that physiology and annotations are in the same order  
        train_physiology_files = sorted(Path(train_physiology_dir).iterdir())
        train_annotation_files = sorted(Path(train_annotations_dir).iterdir())
        # iterate over annotation and physiology files
        for physiology_file_path, annotations_file_path in zip(train_physiology_files, train_annotation_files):
            # make sure that we load corresponding physiology and annotations
            assert physiology_file_path.name == annotations_file_path.name, "Order mismatch"
            # load data from files
            df_physiology = pd.read_csv(physiology_file_path, index_col="time")
            df_annotations = pd.read_csv(annotations_file_path, index_col="time")
            # store data
            storage_dict[fold_dir.name].append((annotations_file_path.name, df_physiology, df_annotations))
    return storage_dict

In [4]:
import matplotlib.pyplot as plt

def plot_data(modality, annotations, physiology, test=False):
    fig = plt.figure(figsize=(16,8))
    fig.patch.set_facecolor('white')
    # plot train physiology with annotations range 
    plt.subplot(211)
    plt.plot(physiology.index, physiology[modality])
    plt.axvspan(annotations.index[0], annotations.index[-1], color='green', alpha=0.3)
    plt.xlim(left=physiology.index[0], right=physiology.index[-1])
    plt.title(f"{test * 'Test' + (not test) * 'Training'} data", fontsize=20)
    plt.ylabel("Signal value", fontsize=16)
    plt.xlabel("Time", fontsize=16)
    plt.xticks(fontsize=14)
    plt.yticks(fontsize=14)
    # plot train annotations
    plt.subplot(212)
    plt.plot(annotations.index, annotations['arousal'], label='arousal - train')
    plt.plot(annotations.index, annotations['valence'], label='valence - train')
    plt.xlim(left=physiology.index[0], right=physiology.index[-1])
    plt.legend(fontsize=14)
    plt.ylabel("Annotation value", fontsize=16)
    plt.xlabel("Time", fontsize=16)
    plt.xticks(fontsize=14)
    plt.yticks(fontsize=14)
    plt.show()

In [5]:
import neurokit2 as nk

def plot_eda(data, fs=1000):
    # Process the raw EDA signal
    preprocess = nk.eda_clean(data, sampling_rate=fs)
    signals, info = nk.eda_process(preprocess, sampling_rate=fs)
    # Extract clean EDA and SCR features
    cleaned = signals["EDA_Phasic"]
    features = [info["SCR_Onsets"], info["SCR_Peaks"], info["SCR_Recovery"]]
    # Visualize SCR features in cleaned EDA signal
    plot = nk.events_plot(features, cleaned, color=['red', 'blue', 'orange'])

---

In [7]:
# specify scenario path
scenario_dir = "../data/scenario_1"

# train data
print("Loading train data")
train = load_data_no_folds(scenario_dir, "train")

# test data
print("Loading test data")
test = load_data_no_folds(scenario_dir, "test")

Loading train data
Loading test data


In [8]:
[name, data, label] = train[2]
name, data, label

('sub_11_vid_11.csv',
          ecg     bvp     gsr     rsp     skt  emg_zygo  emg_coru  emg_trap
 time                                                                      
 0      0.734  36.641  41.599  34.773  25.758     6.549     6.467   148.886
 1      0.754  36.450  41.606  34.773  25.761     6.508     6.548   158.771
 2      0.738  36.583  41.591  34.763  25.761     6.427     6.508   172.897
 3      0.712  36.699  41.653  34.763  25.761     6.385     6.467   185.928
 4      0.672  36.767  41.603  34.754  25.757     6.425     6.467   193.661
 ...      ...     ...     ...     ...     ...       ...       ...       ...
 75046  0.760  36.379  36.496  37.512  25.694     5.480     6.714    27.780
 75047  0.757  36.668  36.508  37.522  25.691     5.400     6.754    27.700
 75048  0.764  36.163  36.430  37.531  25.691     5.480     6.754    27.658
 75049  0.770  36.273  36.430  37.532  25.698     5.440     6.714    27.455
 75050  0.761  36.370  36.437  37.532  25.698     5.399     6.713 

### Look at all the GSR data and their ratings

In [ ]:
for (name, data, label) in train:
    print(name)
    plot_data('gsr', label, data, test=False)
    plt.figure()
    plot_eda(data.gsr)
    plt.show()

### Extract video numbers and order from all loaded data

This allows us to look at data per video to identify potential similarities.

In [ ]:
sub_vid = list()

for (name, data, label) in train:
    parts = name.split('_')
    sub_vid.append(
        (
            int(parts[1]), # get participant number
            int(parts[-1].split('.')[0])) # get video number
    )

df = pd.DataFrame(sub_vid, columns=['participant', 'video'])
videos = sorted(df.video.unique())

In [ ]:
for v in videos:
    print(f"Video {v}")
    for idx in df[df.video == v].index:
        (name, data, label) = train[idx]
        print(name)
        plot_data('gsr', label, data, test=False)
        plt.figure()
        plot_eda(data.gsr)
        plt.show()

---

In [ ]:
# specify scenario path
scenario_dir = "../data/scenario_4"

# train data
print("Loading train data")
load_data_with_folds(scenario_dir, "train")

# test data
print("Loading test data")
load_data_with_folds(scenario_dir, "test")